In [55]:
import tensorflow as tf
from json import loads
import pandas as pd
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from time import time

In [56]:
files = glob("./LOBdata*.csv")
files = [i for i in files if int(i[9:-4]) >= 17]
df  = pd.concat([pd.read_csv(i) for i in files],ignore_index=True)

In [57]:
a = df[df.coin == df.coin.unique()[0]].sort_values('starttime')
print(True in a.label)
X = []
Y = []
for i in range(int(len(a)/60)):
    res = a.iloc[i:i+60]
    matrices = []
    for i in res.matrix:
        matrices.append(loads(i))
    X.append(matrices)
    if res.iloc[0].label == True:
        Y.append([1,0])
    else:
        Y.append([0,1])
Y


False


[[0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

In [26]:
X = []
Y = []
for i in range(len(df)):
    X.append(np.reshape(np.array(loads(df.iloc[i].matrix)),(1,4,7,1)))
    if df.iloc[i].label:
        Y.append([1,0])
    else:
        Y.append([0,1])
# X = tf.convert_to_tensor(X)
# Y = tf.convert_to_tensor(Y)

In [27]:
#split train test 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=int(time()))
X_train, X_test, y_train, y_test = tf.convert_to_tensor(X_train), tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_train), tf.convert_to_tensor(y_test)

In [16]:
X_train.shape

TensorShape([47520, 1, 4, 7, 1])

In [28]:
print(len(df[df['label'] == True].coin.unique()))
print(len(df[df['label'] == False].coin.unique()))

45
45


In [33]:

x = tf.keras.layers.Input((1,4,7,1))
y = tf.keras.layers.Conv2D(64,(2,7),strides=(2,1), activation='relu')(x)
y = tf.keras.layers.Flatten()(y)
y = tf.keras.layers.Dense(32,activation='swish' )(y)
y = tf.keras.layers.Dense(32,activation='swish')(y)
y = tf.keras.layers.Dense(2,activation='sigmoid')(y)

model = tf.keras.Model(inputs=x,outputs=y)
model.summary()



Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1, 4, 7, 1)]      0         
                                                                 
 conv2d_9 (Conv2D)           (None, 1, 2, 1, 64)       960       
                                                                 
 flatten_9 (Flatten)         (None, 128)               0         
                                                                 
 dense_23 (Dense)            (None, 32)                4128      
                                                                 
 dense_24 (Dense)            (None, 32)                1056      
                                                                 
 dense_25 (Dense)            (None, 2)                 66        
                                                                 
Total params: 6,210
Trainable params: 6,210
Non-trainable p

In [34]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
   1e-2,10000, 0.99, name=None
)
adam = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-4,
    amsgrad=False,
    name='Adam'
)
model.compile(optimizer=adam,
              loss='mean_squared_error',
              metrics=['acc',tf.keras.metrics.Precision(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),tf.keras.metrics.Recall(
    thresholds=None, top_k=None, class_id=None, name=None, dtype=None
)])
model.fit(x=X_train,y=y_train,batch_size=32,epochs=200,shuffle=True)

Epoch 1/200
1688/1688 [==============================] - 15s 8ms/step - loss: 0.3645 - acc: 0.6355 - precision_6: 0.6355 - recall_6: 0.6353
Epoch 2/200
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3646 - acc: 0.6354 - precision_6: 0.6354 - recall_6: 0.6354
Epoch 3/200
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3646 - acc: 0.6354 - precision_6: 0.6354 - recall_6: 0.6354
Epoch 4/200
1688/1688 [==============================] - 14s 8ms/step - loss: 0.3646 - acc: 0.6354 - precision_6: 0.6354 - recall_6: 0.6354
Epoch 5/200
1688/1688 [==============================] - 14s 9ms/step - loss: 0.3646 - acc: 0.6354 - precision_6: 0.6354 - recall_6: 0.6354
Epoch 6/200
1688/1688 [==============================] - 14s 8ms/step - loss: 0.3646 - acc: 0.6354 - precision_6: 0.6354 - recall_6: 0.6354
Epoch 7/200
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3646 - acc: 0.6354 - precision_6: 0.6354 - recall_6: 0.6354
Epoch 8/200
1688/168

In [24]:
ans = model.predict(X_test)
# y_test_ = np.argmax(y_test.numpy(),axis=1)
# r = 0
# for i in range(len(y_test)):
#     if y_test_[i] == ans[i] and ans[i] == 0:
#         r+=1

# 0 in ans
ans

array([[0.3361515, 0.6638485],
       [0.3361515, 0.6638485],
       [0.3361515, 0.6638485],
       ...,
       [0.3361515, 0.6638485],
       [0.3361515, 0.6638485],
       [0.3361515, 0.6638485]], dtype=float32)